# Project: Exploration and analysis Airbnb data of Madrid
Author: Luis Carrascosa Cabrera

# Analysing Airbnb data with pandas and plotly


## Introduction
Airbnb is the world’s biggest accommodation-sharing site. Its rapid growth and impact on vacation rentals has generated heated discussions about its effect in several of the world's largest cities. So much that regulations have emerged against the free use of this website:

* Amsterdam: the rental of complete houses is limited to 60 days a year and this will be reduced by half
* Barcelona: short-term rentals must have a license and new licenses are not being issued
* Berlin: owners need a permission to rent 50% or more of their main residence for a short period of time
* London: short-term rentals of entire houses are restricted to 90 days a year
* Palma: the mayor has announced the ban on short-term rentals
* New York: It is usually illegal to rent apartments for 30 consecutive days or less, unless the host is present
* Paris: short-term rentals are limited to 120 days a year
* San Francisco: hosts must register as a company and obtain certificates for short-term rental. The rental of entire properties is limited to 90 days a year
* Singapore: the minimum period of public housing rental is six consecutive months
* Tokyo: home sharing was legalized only in 2017 and is limited to 180 days a year
<div style="text-align: right"><i>Source: https://www.bbc.com/mundo/noticias-45355426</i></div>


Given the [Airbnb dataset of Madrid](http://insideairbnb.com/get-the-data.html), we propose to obtain the following information:

* Which areas have the most Airbnb properties, and which are the most expensive?
* Which amenities increase the price of an Airbnb listing?
* Influence of host features

## Libraries used and version control
I will use numpy, pandas and plotly for the visualization. It's necessary a free mapbox account for drawing maps with plotly

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
from scipy import stats
from plotly.subplots import make_subplots
%matplotlib inline

In [ ]:
mapbox_token_folder = "/home/luis/Documentos/mapbox"
mapbox_access_token = open(f"{mapbox_token_folder}/token").read()
px.set_mapbox_access_token(mapbox_access_token)

### Version control

In [ ]:
print(f'numpy version: {np.__version__}')
print(f'pandas version: {pd.__version__}')
print(f'geopandas version: {gpd.__version__}')

In [ ]:
pip show geojson

In [ ]:
pip show matplotlib

In [ ]:
pip show plotly

## Processing the data
### Import the data

The dataset used for this project comes from Insideairbnb.com. The dataset was scraped on 2019-11-09 and contains information on all Madrid Airbnb listings that were live on the site on that date (20.539)

I will not import free text fields and I will remove the currency symbol from fields with amounts. "smart_location", "zipcode" are redundant having latitude and longitude. "reviews_per_month", "number_of_reviews_ltm" with "number_of_reviews" too. url fields does not add value to the model. I don't import "host_name", "host_location" and "host_about" too. We will not keep any of the host count columns:

In [ ]:
datasetDir = "./dataset"

import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF8') 
conv = locale.localeconv()

fconv_prices = lambda x : locale.atof(x.strip(conv['currency_symbol'])) if x else np.nan 

def fdate_parser(x, dtime_format = "%d-%m-%Y"):
    return datetime.strptime(x, dtime_format) 

In [ ]:
df = pd.read_csv(
    f'{datasetDir}/listings_detailed.csv', 
    sep = ',', 
    index_col = 'id', 
    converters = {
        'price': fconv_prices, 
        'weekly_price': fconv_prices, 
        'monthly_price': fconv_prices,
        'security_deposit': fconv_prices,
        'cleaning_fee': fconv_prices,
        'extra_people': fconv_prices
    },
    parse_dates = ['first_review', 'last_review'],
    usecols = lambda column : column not in [
        'zipcode', 'scrape_id', 'last_scraped', 'name', 'summary', 'space', 'description', 
        'neighborhood_overview', 'notes', 'transit', 'access', 'interaction', 'house_rules', 
        'thumbnail_url', 'medium_url', 'picture_url', 'xl_picture_url', 'host_url', 'host_name', 
        'host_location', 'host_about', 'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood', 
        'host_verifications', 'calendar_last_scraped', 'listing_url', 'smart_location', 'reviews_per_month', 
        'number_of_reviews_ltm', 'host_id', 'host_total_listings_count', 'calculated_host_listings_count', 
        'calculated_host_listings_count_entire_homes', 'calculated_host_listings_count_private_rooms', 
        'calculated_host_listings_count_shared_rooms', 'host_listings_count', 'host_since'
    ]
)

df.head(3)

### Cleaning the features

I will drop columns with more than 85% nulls

In [ ]:
cols_to_drop = set(df.columns[df.isna().mean()>0.85])
print(cols_to_drop)

In [ ]:
df.drop(cols_to_drop, axis='columns', inplace=True)

We only use latitude, longitude and neighbourhood_cleansed location. As we are studying Madrid, the city and country columns do not contribute

In [ ]:
df_gps = df[['latitude', 'longitude']]
df_gps.head()

In [ ]:
df.drop(
    ['neighbourhood', 'neighbourhood_group_cleansed', 'street', 'city', 'state', 'country', 
     'country_code', 'is_location_exact', 'latitude', 'longitude']
    , axis='columns', inplace=True
)

For the number of nights we will only leave "minimum_nights" and "maximum_nights"

In [ ]:
df.drop(
    ['minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights', 
     'maximum_maximum_nights', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm']
    , axis='columns', inplace=True
)

We will change the price column for the price per guest

In [ ]:
df['B_price_guests'] = df['price']/df['guests_included']

df.drop(['price'], axis='columns', inplace=True)
df['B_price_guests'].head()

In [ ]:
df[['B_price_guests']].head()

We'll eliminate prices per guest above quantile 95. Thus we will eliminate possible artificially high prices due to the effect of smart pricing

In [ ]:
outliers = list(df[df['B_price_guests'] >= df['B_price_guests'].quantile(.95)].index)
print(len(outliers))

In [ ]:
df_rev = df.drop(outliers, axis='rows', inplace=False)

box1 = go.Box(y=df['B_price_guests'])
box2 = go.Box(y=df_rev['B_price_guests'])

fig = make_subplots(rows=1, cols=2, subplot_titles=("Original", "95% outliers removal"))
fig.add_trace(box1, row=1, col=1)
fig.add_trace(box2, row=1, col=2)
fig.update_layout(showlegend=False)

fig.show()

Thus we eliminate prices per guest that exceed 5000 and even reach 9000 dollars. Let's compare prices per guest in both dataset

In [ ]:
plotData1 = go.Scatter(
    x=df.index, y=df['B_price_guests'], 
    mode='markers',
    marker=dict(
        color='LightSkyBlue',
        size=2
    )
)

plotData2 = go.Scatter(
    x=df_rev.index, y=df_rev['B_price_guests'], 
    mode='markers',
    marker=dict(
        color='LightSkyBlue',
        size=2
    )
)

layout = go.Layout(
    title={'x': 0.5},
    xaxis={'title': 'id'},
    yaxis={'title': 'B_price_guests'},
    margin=go.layout.Margin(l=0, r=0, t=0, b=0)
)

fig = make_subplots(rows=2, cols=1, subplot_titles=("Madrid airbnb data", 
                                                    "Madrid airbnb data without 95% outliers"))
fig.add_trace(plotData1, row=1, col=1)
fig.add_trace(plotData2, row=2, col=1)
fig.update_layout(showlegend=False)

fig.update_layout(title_text='Price per guest')

fig.show()

We don't lose the two concentrations near the ids 20M and 35M.
<br>Let's represent each column to review them

In [ ]:
df_temp = df_rev.replace({'f': 0, 't': 1}, inplace=False)
df_temp.hist(figsize=(20,20));

Columns with only one category are deleted

In [ ]:
df_rev.drop(['has_availability', 'host_has_profile_pic', 'is_business_travel_ready', 'require_guest_phone_verification', 
 'require_guest_profile_picture', 'requires_license'], 
        axis='columns', inplace=True)

## Answering questions
* Which areas have the most Airbnb properties, and which are the most expensive?
<br> We need the gps coordinates

In [ ]:
df_full_map = df_gps.copy().join(df_rev.loc[:, ['B_price_guests', 'neighbourhood_cleansed']], on = 'id', how = 'inner')
df_full_map.head()

In [ ]:
fig = px.scatter_mapbox(
    df_full_map, lat = "latitude", lon = "longitude",
    color = "B_price_guests", color_continuous_scale = px.colors.cyclical.IceFire,
    zoom = 10
)

fig.update_layout(
    margin=dict(l=0, r=0, t=0, b=0),
    showlegend=True,
    autosize=True
)

fig.show()

We need to import the geojson data file and get the representatives points to annotate in the map

In [ ]:
df_madrid_map = gpd.read_file(f'{datasetDir}/neighbourhoods.geojson')
crs = {"init": "epsg:4326"}  

df_madrid_map = df_madrid_map.rename(columns={"neighbourhood": "neighbourhood_cleansed"})
df_madrid_map = df_madrid_map.set_index('neighbourhood_cleansed')
df_madrid_map = df_madrid_map.drop('neighbourhood_group', axis=1)

df_madrid_map['rep_point'] = df_madrid_map.geometry.apply(lambda x: x.representative_point().coords[:][0])

We group by "neighbourhood_cleansed" and we generate a GeoDataFrame to draw a color map (I can't do it with Plotly)

In [ ]:
df_agg_neighbourhood = df_full_map.groupby('neighbourhood_cleansed').size().to_frame()
df_agg_neighbourhood.rename(columns={0: 'number_of_listings'}, inplace=True)
df_agg_neighbourhood['median_price'] = df_full_map.groupby('neighbourhood_cleansed').B_price_guests.median().values
df_agg_neighbourhood['mean_price'] = round(df_full_map.groupby('neighbourhood_cleansed').B_price_guests.mean(), 2).values

geo_df_agg_neighbourhood = gpd.GeoDataFrame(
    df_agg_neighbourhood.join(df_madrid_map, on = 'neighbourhood_cleansed', how = 'inner'), 
    crs = crs
) 

geo_df_agg_neighbourhood.sort_values(by='number_of_listings')

We define a function and plot number of listings, median and mean
### Listings number

In [ ]:
def region_plot(df_plot, column, rep_point_col = 'rep_point', titleName = '', legend_label = '', 
                fontsize=10, annonations = False, aspect = 1., figsize=(25, 25), dpi = 100):
    fig, ax = plt.subplots(figsize=figsize, dpi = dpi)

    df_plot.plot(
        alpha = 1, cmap='Blues', edgecolor = "black", ax = ax, 
        column=column,
        legend=True,
        legend_kwds={'label': legend_label, 'orientation': "vertical"}
    )

    if annonations != False:
        for idx, row in df_plot.iterrows():
            ax.annotate(
                s=idx, 
                xy=row[rep_point_col],
                horizontalalignment='center'
            )
        
    ax.axis('off')
    ax.set_title(titleName, fontsize=fontsize)

    ax.set_aspect(aspect=aspect)
    ax.margins(0)
    ax.apply_aspect()
    
    plt.show()

In [ ]:
region_plot(
    geo_df_agg_neighbourhood[geo_df_agg_neighbourhood['number_of_listings']>50], 
    column = 'number_of_listings', titleName = 'Neighborhoods with 50 listings or greater', 
    fontsize=20, annonations = True, aspect=4./3, legend_label = 'Number of listings', figsize=(20, 15))

In [ ]:
region_plot(
    geo_df_agg_neighbourhood[geo_df_agg_neighbourhood['number_of_listings']>100], 
    column = 'number_of_listings', titleName = 'Neighborhoods with 100 listings or greater', 
    fontsize=20, annonations = True, aspect = 4./3, legend_label = 'Number of listings', figsize=(18, 12)
)

In [ ]:
region_plot(
    geo_df_agg_neighbourhood[geo_df_agg_neighbourhood['number_of_listings']>500], 
    column = 'number_of_listings', titleName = 'Neighborhoods with 500 listings or greater', 
    fontsize=12, annonations = True, aspect = 4./3, legend_label = 'Number of listings', figsize=(8, 5)
)

In [ ]:
geo_df_agg_neighbourhood[geo_df_agg_neighbourhood['number_of_listings']>500][['number_of_listings', 'median_price']]

The largest number of houses is in the center
### Price per guest included
<br> Mean or median? There are listings with a high price and only one guest included, but its "accomodates" is high. Let's use median to soften those anomalies

In [ ]:
region_plot(
    geo_df_agg_neighbourhood, 
    column = 'median_price', titleName = 'Median price per guest', 
    fontsize=14, annonations = False, aspect = 4./3, legend_label = 'Median price per guest', figsize=(25, 15)
)

There are peripheral areas with a high median. Let's filter by min 50$

In [ ]:
region_plot(
    geo_df_agg_neighbourhood[geo_df_agg_neighbourhood['median_price']>40.], 
    column = 'median_price', titleName = 'Neighborhoods with median price per guest greater than 50.0$)', 
    fontsize=14, annonations = True, aspect = 4./3, legend_label = 'Median price per guest', figsize=(25, 15)
)

In [ ]:
geo_df_agg_neighbourhood[geo_df_agg_neighbourhood['median_price']>50][['number_of_listings', 'median_price']]

The areas with the highest median price are no longer all concentrated in the city center. The "Cuatro Vientos" area is notable, surely there is a high demand due to the air base sited there. However it has little offer, which would explain these high prices

In [ ]:
df_full_map[df_full_map.neighbourhood_cleansed == 'Cuatro Vientos']

## Which amenities increase the price of an Airbnb listing?
We will draw a barchart for every amenity group to compare number of listings versus median price.

**amenities**
<br>Additional features in the property, e.g. whether it has a TV or parking. This data is a list and we need to extract a list of all posibles amenities 

In [ ]:
amenities_list = set()
for index, value in df_rev.amenities.items():
    value = value.replace('{', '').replace('"', '').replace('}', '')
    amenities_list = amenities_list.union(set(value.split(sep=',')))
    
print(amenities_list)

I define the following groupings of amenities:

In [ ]:
amenities_regexprs = [
r'(?P<B_amenities_pets>Pets allowed|Other pet|Dog|Cat|Pets live on this property)',
r'(?P<B_amenities_kitchen>Kitchen|Kitchenette|Full kitchen)',
r'(?P<B_amenities_high_kitchen>Warming drawer|Wine cooler|Steam oven|Double oven)',
r'(?P<B_amenities_high_bedroom>Firm mattress|Memory foam mattress|Pillow-top mattress)',
r'(?P<B_amenities_outdoor>BBQ grill|Terrace|Outdoor seating|Patio or balcony|Hammock|Garden or backyard|Balcony|Sun loungers)',
r'(?P<B_amenities_family_friendly>Family\/kid friendly|Changing table|Baby bath|Crib|Children’s books and toys|Pack ’n Play\/travel crib|Fireplace guards|Babysitter recommendations|Window guards|Children’s dinnerware|Stair gates|Table corner guards|Outlet covers|Baby monitor|High chair)',
r'(?P<B_amenities_home_appliances>Washer \/ Dryer|Refrigerator|Mini fridge|Gas oven|Cooking basics|Dryer|Convection oven|Microwave|Washer|Oven|Dishes and silverware|Iron|Dishwasher)',
r'(?P<B_amenities_high_electronics>HBO GO|Smart TV|Netflix|Projector and screen|Game console|Amazon Echo)',
r'(?P<B_amenities_normal_electronics>DVD player|Printer|TV|Cable TV|Sound system|High-resolution computer monitor)',
r'(?P<B_amenities_special_accessibility>Fixed grab bars for shower|Accessible-height bed|Extra space around bed|Pool with pool hoist|Electric profiling bed|Accessible-height toilet|Wide doorway to guest bathroom|Ground floor access|Single level home|Wheelchair accessible|Disabled parking spot|Shower chair|No stairs or steps to enter|Step-free shower|Bathtub with bath chair|Wide entrance|Fixed grab bars for toilet|Wide entrance for guests|Flat path to guest entrance|Mobile hoist|Wide clearance to shower|Wide hallways|Wide entryway)',
r'(?P<B_amenities_breakfast_appliances>Espresso machine|Hot water kettle|Coffee maker)',
r'(?P<B_amenities_gym>Gym|Exercise equipment)',
r'(?P<B_amenities_pool>Shared pool|Pool)',
r'(?P<B_amenities_heating>Heated towel rack|Heating|Heated floors|Stove)',
r'(?P<B_amenities_other>Other|Essentials|Breakfast table|Beach essentials|Buzzer\/wireless intercom|Formal dining area|Ceiling fan|Day bed|Room-darkening shades|Laptop friendly workspace|Standing valet)',
r'(?P<B_amenities_views>Beach view|Ski-in/Ski-out|Waterfront|Beachfront|Beachfront)',
r'(?P<B_amenities_internet>Ethernet connection|Wifi|Pocket wifi|Internet)',
r'(?P<B_amenities_bathroom_basics>Hot water|Walk-in shower|Bathtu|Soaking tub|Handheld shower head|Bath towel|Bidet|Shampoo|Rain shower|Body soap|toilet|Bathroom essentials|Toilet paper|Hair dryer)',
r'(?P<B_amenities_cleaning_before_checkout>Cleaning before checkout)',
r'(?P<B_amenities_bedroom_basics>Bedroom comforts|Extra pillows and blankets|Bed linens|Hangers)',
r'(?P<B_amenities_security>Lockbox|Lock on bedroom door)',
r'(?P<B_amenities_indoor_fireplace>Indoor fireplace)',
r'(?P<B_amenities_staff>Building staff|Doorman)',
r'(?P<B_amenities_smoking>Smoking allowed)',
r'(?P<B_amenities_high_bathroom>Jetted tub|Touchless faucets|Hot tub)',
r'(?P<B_amenities_cleaning_before_checkout>Cleaning before checkout)',    
r'(?P<B_amenities_proximity_host>Host greets you)',
r'(?P<B_amenities_general_accesibility>Well-lit path to entrance|Elevator)',
r'(?P<B_amenities_free_parking>Free street parking|Free parking on premises)',  
r'(?P<B_amenities_paid_parking>Paid parking on premises|Paid parking off premises)',
r'(?P<B_amenities_air_conditioning>Central air conditioning|Air conditioning)',
r'(?P<B_amenities_privacy>Private living room|Private bathroom|Private entrance|En suite bathroom)',
r'(?P<B_amenities_safety>Fire extinguisher|Carbon monoxide detector|Smoke detector|Safety card|First aid kit)',
r'(?P<B_amenities_luggage>Mudroom|Luggage dropoff allowed)',    
r'(?P<B_amenities_ev_charger>EV charger)',     
r'(?P<B_amenities_breakfast>Breakfast)',
r'(?P<B_amenities_self_check>Keypad|24-hour check-in|Self check-in|Smart lock)',    
r'(?P<B_amenities_suitable_events>Suitable for events)',
r'(?P<B_amenities_long_term>Long term stays allowed)'   
]

Iterating through the list of regular expressions we will build a dataframe with the groups of defined things that we will add to obtain the totals of each group. These regular expressions could be concatenated to obtain such dataframe but in this way pandas does not find all the matches

In [ ]:
def_amenities = df_rev.amenities.str.extract(amenities_regexprs[0], expand=True)

for reg_expr in amenities_regexprs[1:]:
    col_name = reg_expr.split('<')[1].split('>')[0]
    def_amenities[col_name] = df_rev.amenities.str.extract(reg_expr, expand=False)
    
serie_amenities = def_amenities.notna().sum()/len(df_rev)*100.0
round(serie_amenities, 2)

We will remove those that offer more than 90% of ads and those that are below 10%

In [ ]:
low_frequent_cols = list(serie_amenities[serie_amenities<10.0].index)
high_frequent_cols = list(serie_amenities[serie_amenities>90.0].index)

def_amenities.drop(low_frequent_cols, axis=1, inplace=True)
def_amenities.drop(high_frequent_cols, axis=1, inplace=True)

print(f"high_frequent_cols: {high_frequent_cols}\n\nlow_frequent_cols: {low_frequent_cols}")

In [ ]:
round(def_amenities.notna().sum()/len(df_rev)*100.0, 2)

If it is null we change it to an "f". If it is not for a "t"

In [ ]:
for col in list(def_amenities.columns):
    def_amenities.loc[def_amenities[col].notna(), col] = 't'
    def_amenities.loc[def_amenities[col].isna(), col] = 'f'
    
def_amenities.head()

Let's draw a histogram to see the columns we have obtained by extracting the information from "amenities"

In [ ]:
df_temp = def_amenities.replace({'f': 0, 't': 1}, inplace=False)
df_temp.hist(figsize=(15,15));

We join both dataframe and drop the "amenities" column

In [ ]:
df_rev.drop(['amenities'], axis=1, inplace=True)
df_full = df_rev.join(def_amenities, on = 'id', how = 'inner')
df_full.head()

In [ ]:
dict_amenities = {
    x:x.replace('B_amenities_', '').replace('_', ' ').upper() 
    for x in df_full.filter(like='B_amenities_').columns.to_list()
}

def plot_barchart(df, dict_columns = dict_amenities, width=800, height=300):   
    for col in list(dict_columns):
        s_1 = df.groupby(col).size()
        s_2 = df.groupby(col)['B_price_guests'].median()

        fig = make_subplots(rows=1, cols=2, shared_xaxes=True, 
                    subplot_titles=("Listings number","Median price ($)"))
        
        fig.add_trace(go.Bar(
            x=s_1.index, y=s_1.values, marker_color='indianred',
            text=s_1.values, textposition='auto'), row=1, col=1)

        fig.add_trace(go.Bar(
            x=s_2.index, y=s_2.values, marker_color='seagreen',
            text=s_2.values, textposition='auto'), row=1, col=2)

        fig.update_layout(title_text=dict_columns[col], showlegend=False, width=width, height=height)
        fig.show()

In [ ]:
plot_barchart(df_full)

These are the things that seem to influence (positively) the median price:
* Normal electronics: DVD player, Printer, TV, Cable TV, Sound system, High-resolution computer monitor
* Air conditioning: Central air conditioning, Air conditioning

What if only the neighborhoods with the highest number of listings are reviewed?

In [ ]:
many_listings = geo_df_agg_neighbourhood[geo_df_agg_neighbourhood['number_of_listings']>500].index.to_list()
many_listings

In [ ]:
df_full_many = df_full.loc[df_full.neighbourhood_cleansed.isin(many_listings)].copy()
plot_barchart(df_full_many)

Others amenities begin to influence the price because these neighborhoods are in the center of the city.
* Normal electronics: DVD player, Printer, TV, Cable TV, Sound system, High-resolution computer monitor
* Air conditioning: Central air conditioning, Air conditioning
* General accesibility (little): Well-lit path to entrance, Elevator
<br> In the center there are old buildings without an elevator. Maybe for that reason customers look for the house to have an elevator in these neighborhoods
* Free parking: Free street parking, Free parking on premises
<br> It's difficult to park the car in the central neighborhoods
* Privacy: Private living room, Private bathroom, Private entrance, En suite bathroom

## Influence of host features
We want to analyze how categorical variables related to the host affect the median price. We'll use another histograms
* 'host_response_time'
* 'host_response_rate'
* 'host_is_superhost'
* 'host_identity_verified'

**host_response_time** - average amount of time the host takes to reply to messages
<br>There is some possible values and nans. This nans we will impute these to a new value: unknown

In [ ]:
df_full.host_response_time.fillna("unknown", inplace=True)

In [ ]:
df_full.host_response_time.value_counts()

**host_response_rate**
<br>Analogous case but for which we will group in a categorical variable: "unknown", \[0-49%\], \[50-89%\], \[90-99%\] y 100%

In [ ]:
df_full.host_response_rate = df_full.host_response_rate.str[:-1].astype('float64').fillna(-1)

In [ ]:
df_full.host_response_rate = pd.cut(df_full.host_response_rate, 
                                   bins=[-2, 0, 50, 90, 99, 100], 
                                   labels=['unknown', '0-49%', '50-89%', '90-99%', '100%'], 
                                   include_lowest=True)

In [ ]:
df_full.host_response_rate = df_full.host_response_rate.str[:-1].astype('float64')

df_full.host_response_rate = pd.cut(df_full.host_response_rate, 
                                   bins=[0, 50, 90, 99, 100], 
                                   labels=['0-49%', '50-89%', '90-99%', '100%'], 
                                   include_lowest=True)

df_full.host_response_rate = df_full.host_response_rate.astype('str')
df_full.host_response_rate.replace('nan', 'unknown', inplace=True)

df_full.host_response_rate.value_counts()

**host_is_superhost** - whether or not the host is a superhost, which is a mark of quality for the top-rated and most experienced hosts
<br>There are only 12 nans. We will review the rest of columns "host_"

In [ ]:
df_full[df_full.host_is_superhost.isna()].filter(like='host_')

"host_is_superhost", "host_listings_count" and "host_identity_verified" has the same nulls. We drop these listings

In [ ]:
df_full.drop(df_full[df_full.host_is_superhost.isna()].index, inplace = True)

In [ ]:
df_full[df_full.host_is_superhost.isna()].filter(like='host_')

**host_identity_verified** - whether or not the host has been verified with id

In [ ]:
df_full.host_identity_verified.isna().sum()

In [ ]:
df_full.host_identity_verified.value_counts()

In [ ]:
host_cols = df_full.filter(like='host_').columns.to_list()
dict_host = {key:key.replace('_', ' ').upper() for key in host_cols}
dict_host

In [ ]:
plot_barchart(df_full, dict_host)

**'host_response_time' and 'host_response_rate'**<br>
The "worst" categories are those with the highest median price. It seems that this feature does not "cause" the price, but is a consequence of the price above the median. The host knows that he has a well-valued property in Air Bnb and does not give such exhaustive attention to potential clients

**'host_is_superhost' and 'host_identity_verified'**<br>
The definition of 'host_is_superhost', according to [Air Bnb](https://www.airbnb.co.uk/help/article/828/what-is-a-superhost?_set_bev_on_new_domain=1579644646_eBD%2Bqp6saBYIE3Z7) 
> "Superhosts are experienced hosts who provide a shining example for other hosts, and extraordinary experiences for their guests"

The definition of 'host_identity_verified', according to [Air Bnb](https://www.airbnb.co.uk/help/article/1237/how-does-it-work-when-airbnb-verifies-your-identity?_set_bev_on_new_domain=1579644646_eBD%2Bqp6saBYIE3Z7)
> At Airbnb, we’re always working on making our community as secure as possible for everyone. That’s why, before booking a home or experience, or becoming a host, we may ask for a government ID or ask you to confirm your legal name and add your address

In both cases there is no effect on the price